In [1]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import zipfile, os


In [2]:
zip_path = "/content/router_dataset.zip"
extract_path = "/content/router_dataset/"

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall("/content/")

print("Extracted folders:", os.listdir("/content/router_dataset"))


Extracted folders: ['pest', 'leaf', 'background', 'fruit']


In [3]:
IMG_SIZE = (224, 224)
BATCH = 32

train_gen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
)

train_data = train_gen.flow_from_directory(
    "/content/router_dataset",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_data = train_gen.flow_from_directory(
    "/content/router_dataset",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

print("\nClass mapping:", train_data.class_indices)


Found 679 images belonging to 4 classes.
Found 169 images belonging to 4 classes.

Class mapping: {'background': 0, 'fruit': 1, 'leaf': 2, 'pest': 3}


In [4]:
base = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)
base.trainable = False  # Freeze weights

x = layers.GlobalAveragePooling2D()(base.output)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
output = layers.Dense(4, activation="softmax")(x)

model = Model(base.input, output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,468 (9.24 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [5]:
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=2)
]

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=callbacks
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 129s 5s/step - accuracy: 0.5420 - loss: 1.1016 - val_accuracy: 0.8462 - val_loss: 0.3908 - learning_rate: 8.0000e-04
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 72s 3s/step - accuracy: 0.9404 - loss: 0.2081 - val_accuracy: 0.8698 - val_loss: 0.3200 - learning_rate: 8.0000e-04
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.9711 - loss: 0.1314 - val_accuracy: 0.8876 - val_loss: 0.2737 - learning_rate: 8.0000e-04
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - accuracy: 0.9784 - loss: 0.0759 - val_accuracy: 0.8935 - val_loss: 0.2547 - learning_rate: 8.0000e-04
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - accuracy: 0.9822 - loss: 0.0523 - val_accuracy: 0.9290 - val_loss: 0.1991 - learning_rate: 8.0000e-04
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.9916 - loss: 0.0438 - val_accuracy: 0.9053 - val_loss: 0.2670 - learning_rate: 8.0000e-04
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.9887 - loss

In [6]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Predict all validation images
val_preds = model.predict(val_data)
true = val_data.classes
pred = np.argmax(val_preds, axis=1)

print(classification_report(true, pred, target_names=list(train_data.class_indices.keys())))

cm = confusion_matrix(true, pred)
cm


6/6 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step
              precision    recall  f1-score   support

  background       0.85      0.95      0.90        42
       fruit       1.00      0.93      0.96        42
        leaf       0.93      1.00      0.97        42
        pest       0.92      0.81      0.86        43

    accuracy                           0.92       169
   macro avg       0.93      0.92      0.92       169
weighted avg       0.93      0.92      0.92       169



array([[40,  0,  0,  2],
       [ 0, 39,  2,  1],
       [ 0,  0, 42,  0],
       [ 7,  0,  1, 35]])

In [7]:
with zipfile.ZipFile("router_new.zip", 'r') as z:
    z.extractall("/content/router_new/")

In [9]:
src = "/content/router_new/router_new/pest_new/"
dst = "/content/router_dataset/pest/"


In [10]:
import os, shutil

count = 0
for img in os.listdir(src):
    shutil.copy(os.path.join(src, img), dst)
    count += 1

print("Total NEW pest images added:", count)


Total NEW pest images added: 170


In [11]:
os.listdir("/content/router_new/router_new/")


['pest_new', 'background_new']

In [12]:
os.listdir("/content/router_new/router_new/pest_new/")


['img_35806.jpg',
 'img_35935.jpg',
 'img_35590.jpg',
 'img_35682.jpg',
 'img_35662.jpg',
 'img_35610.jpg',
 'img_35803.jpg',
 'img_35960.jpg',
 'img_35713.jpg',
 'img_35931.jpg',
 'img_35925.jpg',
 'img_35956.jpg',
 'img_35802.jpg',
 'img_35779.jpg',
 'img_35735.jpg',
 'img_35589.jpg',
 'img_35832.jpg',
 'img_35941.jpg',
 'img_35729.jpg',
 'img_35734.jpg',
 'img_35721.jpg',
 'img_35716.jpg',
 'img_35833.jpg',
 'img_35929.jpg',
 'img_35725.jpg',
 'img_35815.jpg',
 'img_35980.jpg',
 'img_35755.jpg',
 'img_35809.jpg',
 'img_35669.jpg',
 'img_35764.jpg',
 'img_35593.jpg',
 'img_35585.jpg',
 'img_35671.jpg',
 'img_35664.jpg',
 'img_35733.jpg',
 'img_35937.jpg',
 'img_35782.jpg',
 'img_35588.jpg',
 'img_35927.jpg',
 'img_35661.jpg',
 'img_35933.jpg',
 'img_35728.jpg',
 'img_35715.jpg',
 'img_35723.jpg',
 'img_35949.jpg',
 'img_35775.jpg',
 'img_35587.jpg',
 'img_35977.jpg',
 'img_35928.jpg',
 'img_35594.jpg',
 'img_35727.jpg',
 'img_35777.jpg',
 'img_35961.jpg',
 'img_35612.jpg',
 'img_3574

In [13]:
import hashlib, os

def remove_duplicates(folder):
    hashes = {}
    removed = 0
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        try:
            with open(path, 'rb') as f:
                filehash = hashlib.md5(f.read()).hexdigest()
            if filehash in hashes:
                os.remove(path)
                removed += 1
            else:
                hashes[filehash] = filename
        except:
            pass
    print("Duplicates removed:", removed)

remove_duplicates("/content/router_dataset/pest/")


Duplicates removed: 1


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224,224)
BATCH = 32

datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    "/content/router_dataset",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    subset="training",
    class_mode="categorical",
    shuffle=True
)

val_data = datagen.flow_from_directory(
    "/content/router_dataset",
    target_size=IMG_SIZE,
    batch_size=BATCH,
    subset="validation",
    class_mode="categorical",
    shuffle=False
)

print("Class mapping:", train_data.class_indices)


Found 814 images belonging to 4 classes.
Found 203 images belonging to 4 classes.
Class mapping: {'background': 0, 'fruit': 1, 'leaf': 2, 'pest': 3}


In [15]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 96s 4s/step - accuracy: 0.9821 - loss: 0.0698 - val_accuracy: 0.9803 - val_loss: 0.0879
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.9991 - loss: 0.0281 - val_accuracy: 0.9704 - val_loss: 0.0887
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 0.9945 - loss: 0.0429 - val_accuracy: 0.9803 - val_loss: 0.0727
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 1.0000 - loss: 0.0208 - val_accuracy: 0.9803 - val_loss: 0.0757
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 72s 3s/step - accuracy: 1.0000 - loss: 0.0171 - val_accuracy: 0.9704 - val_loss: 0.0765


In [16]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

val_preds = model.predict(val_data)
true = val_data.classes
pred = np.argmax(val_preds, axis=1)

print(classification_report(true, pred))
confusion_matrix(true, pred)


7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step
              precision    recall  f1-score   support

           0       0.93      0.98      0.95        42
           1       0.98      0.98      0.98        42
           2       0.95      1.00      0.98        42
           3       1.00      0.95      0.97        77

    accuracy                           0.97       203
   macro avg       0.97      0.98      0.97       203
weighted avg       0.97      0.97      0.97       203



array([[41,  1,  0,  0],
       [ 0, 41,  1,  0],
       [ 0,  0, 42,  0],
       [ 3,  0,  1, 73]])

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("router_model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpc4cgdspp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  140498346370704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346371280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346373968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346373584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346372432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346374160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346372624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346374736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346374352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140498346372240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14049834637089

In [18]:
import json

class_map = {v:k for k,v in train_data.class_indices.items()}

with open("router_class_map.json", "w") as f:
    json.dump(class_map, f, indent=4)
